Ce notebook a pour but d'ajouter des colonnes à X_train ou X_test à partir de fichiers de données déjà nettoyés.

In [0]:
import numpy as np
import pandas as pd
from math import isnan

# Importation des fichiers originaux

In [0]:
X_train_url = "https://raw.githubusercontent.com/BenjaminBcmp/DataChallengeGenerali/master/data/X_train.csv"
X_test_url = "https://raw.githubusercontent.com/BenjaminBcmp/DataChallengeGenerali/master/data/X_test.csv"
y_train_url = "https://raw.githubusercontent.com/BenjaminBcmp/DataChallengeGenerali/master/data/y_train_saegPGl.csv"
X_train = pd.read_csv(X_train_url, header=0, index_col=0)
X_test = pd.read_csv(X_test_url, header=0, index_col=0)
y_train = pd.read_csv(y_train_url, header=0, index_col=0)

Données manquantes pour train et test :

In [0]:
for col in X_train.columns:
    print(col, np.sum(X_train[col].isna()))

Identifiant 0
ft_2_categ 0
EXPO 0
ft_4_categ 0
ft_5_categ 0
ft_6_categ 0
ft_7_categ 0
ft_8_categ 0
ft_9_categ 0
ft_10_categ 0
ft_11_categ 0
ft_12_categ 0
ft_13_categ 0
ft_14_categ 0
ft_15_categ 0
ft_16_categ 0
ft_17_categ 0
ft_18_categ 0
ft_19_categ 0
superficief 119
ft_21_categ 0
ft_22_categ 1236
ft_23_categ 0
ft_24_categ 0
Insee 115


In [0]:
for col in X_test.columns:
    print(col, np.sum(X_test[col].isna()))

Identifiant 0
ft_2_categ 0
EXPO 0
ft_4_categ 0
ft_5_categ 0
ft_6_categ 0
ft_7_categ 0
ft_8_categ 0
ft_9_categ 0
ft_10_categ 0
ft_11_categ 0
ft_12_categ 0
ft_13_categ 0
ft_14_categ 0
ft_15_categ 0
ft_16_categ 0
ft_17_categ 0
ft_18_categ 0
ft_19_categ 0
superficief 42
ft_21_categ 0
ft_22_categ 402
ft_23_categ 0
ft_24_categ 0
Insee 39


Nous allons traiter les données manquantes de `ft_22_categ` comme une nouvelle catégorie. Pour le train, nous supprimons les autres lignes où il manque des données. Pour le test, nous remplacerons les autres données manquantes (dans `superficief` ainsi que celles que nous allons rajouter) par la moyenne ou la médiane de la variable.

# Quelques fonctions pour ajouter des colonnes à notre jeu de données

## Données manquantes

In [0]:
def is_nan(val):
    if type(val) is float and isnan(val):
        return True
    return False

def replace_missing_by_mean(col):
    """Données manquantes : -1
    On les remplace par la moyenne de la colonne"""
    moy = np.mean(col[col!=-1])
    col[col==-1] = moy
    return col

def replace_missing_by_med(col):
    """Données manquantes : -1
    On les remplace par la médiane de la colonne"""
    moy = np.median(col[col!=-1])
    col[col==-1] = moy
    return col

## Chômage

In [0]:
def ajout_chomage(X, chomage):
    """Renvoie la colonne contenant le taux de chômage par année et code Insee"""
    departements = np.array(chomage['Code'])
    #On ajoute le chomage quand l'insee est spécifié, -1 sinon
    colonne = np.array([chomage[str(annee)][departements==insee[:2]].values[0] \
                     if not is_nan(insee) else -1 for annee,insee in zip(X.ft_2_categ, X.Insee) ])
    #On remplace les 0 par la moyenne du chomage
    colonne = replace_missing_by_mean(colonne)
    return colonne

## Revenus

On commence par écrire une fonction qui prédit (estime) le revenu médian en 2016 (données non disponibles). On choisit de prendre la moyenne des revenus médians des deux années précédentes :

In [0]:
def pred_revenu_2016(code, revenus, revenus_dep):
    """Prédit le revenu médian en 2016 pour le code Insee.
    Prédiction = moyenne des revenus médians des deux années précédentes pour le
    code INSEE quand ils sont disponibles, sinon on utilise les revenus du 
    département"""
    rev14, rev15 = revenus[2], revenus[3]
    if code in np.array(rev14.CODGEO) and code in np.array(rev15.CODGEO):
        return (rev14.MED[rev14.CODGEO==code].values[0] + 
                rev15.MED[rev15.CODGEO==code].values[0] ) / 2
    else:
        revdep14, revdep15 = revenus_dep[2], revenus_dep[3]
        return (revdep14.MED[revdep14.CODGEO==code[:2]].values[0] + 
                revdep15.MED[revdep15.CODGEO==code[:2]].values[0] ) / 2

Puis on écrit la fonction qui renvoie la colonne des revenus médians, en discriminant suivant si le code Insee est présent dnas la liste des codes Insee des données "revenus". Dans le cas où le code Insee n'existe pas (c'est le cas pour une soixantaine de lignes), on prend le revenu médian du département :

In [0]:
def ajout_revenu(X, revenus, revenus_dep):
    """Renvoie la colonne contenant le revenu médian pour les années où l'on 
    dispose des données, et une prédiction de ce revenu pour les autres années"""
    codes = [np.array(revenus[annee-2012].CODGEO) for annee in range(2012,2016)]
    n = len(X)
    rev_med = np.zeros(n)
    for k in range(n):
        annee = int(X.ft_2_categ[k])
        if is_nan(X.Insee[k]):
            rev_med[k] = -1
        elif annee == 2016:
            rev_med[k] = pred_revenu_2016(X.Insee[k], revenus, revenus_dep)
        elif X.Insee[k] not in codes[annee-2012]:
            rev_med[k] = revenus_dep[annee-2012].MED[revenus_dep[annee-2012].CODGEO==X.Insee[k][:2]]
        else:
            rev_med[k] = revenus[annee-2012].MED[revenus[annee-2012].CODGEO==X.Insee[k]]
     
    #On remplace les valeurs manquantes
    rev_med = replace_missing_by_mean(rev_med)

    return rev_med

# Association des données supplémentaires

In [0]:
def prep_train(X, y, path):
    #Importation des fichiers de données
    cat = path +'/catastrophes naturelles/'
    crim = path +'criminalité/'
    catnat = pd.read_csv(cat+"catnat_propre.csv", header=0,sep=';')
    crimes = pd.read_excel(crim+"crimes_propre.xlsx", header=0,sep=';')
    pprn = pd.read_excel(cat+"pprn_propre.xlsx", header=0)
    pluie = pd.read_excel(cat+"pluie.xlsx", header=0)
    chomage = pd.read_csv(path+"chomage/chomage_propre.csv", header=0, index_col=0)
    log_sociaux=pd.read_excel(path+"log_sociaux.xls",header=0,sep='')
    revenus = [] #Liste de DataFrames
    revenus_dep = [] #Revenus par départements
    for annee in range(2012,2016):
        revenus.append(pd.read_csv(path+"revenus/revenus_%s_propre.csv"%(annee), header=0, index_col=0))
        revenus_dep.append(pd.read_csv(path+"revenus/revenus_%s_dep_propre.csv"%(annee), header=0, index_col=0))

    ##On supprime les NaN (hors ft_22_categ), on réindexe X et on sauvegarde les indices supprimés
    X = X.dropna(axis=0, subset=['superficief', 'Insee'])
    indices = X.index.values
    X.index = range(len(X))
    
    #On supprime les mêmes éléments dans y
    y = y.loc[indices,:]
  
    X_ajout = X.copy()
    Catnat = []
    Crim = []
    Cpprn = []
    Cpluie = []
    Cft22 = []
    Clog_sociaux=[]
    
    ##Parcours des lignes pour crimes et catastrophes naturelles
    for x in X_ajout.itertuples():
    
        ##Données sur le nombre de jour de catastrophes naturelles
        a=catnat[x.Insee==catnat.Insee]
        a=a[x.ft_2_categ==a.annee]
        ##On somme les différents arrêtés simulatannés pour la même zone
        Catnat+=[a.num_risque_jo.sum()]
    
        ##Données sur les PPRN
        a=pprn[x.Insee[:2]==pprn.cod_commune]
        a=a[x.ft_2_categ==a.dat_mise_a_enquete_deb]
        Cpprn+=[len(a)]
    
        ##Données sur le nombre de jour avec plus de 1m de précipitation
        a=pluie[x.Insee[:2]==pluie.departement]
        Cpluie+=[a[x.ft_2_categ].iloc[0]]
    
        ##Données sur le taux de criminalité
        a=crimes[x.Insee[:2]==crimes.departement]
        a=a[x.ft_2_categ==a.annee]
        Crim+=[a.criminalite.iloc[0]]
        
        #Données manquantes de ft_22_categ
        if is_nan(x.ft_22_categ):
            Cft22 += [-1]
        else:
            Cft22 += [x.ft_22_categ]
            
        #Données sur le logement sociaux
        a=log_sociaux[x.Insee[:2]==log_sociaux.Dep]
        Clog_sociaux+=[a.Nb_log_sociaux.iloc[0]]
        
    ##Chomage
    Cchomage = ajout_chomage(X_ajout, chomage)
    
    ##Revenus
    Crevenus = ajout_revenu(X, revenus, revenus_dep)
    
    #Ajout des colonnes dans le DataFrame
    X_ajout['pluie'] = Cpluie
    X_ajout['catnat'] = Catnat
    X_ajout['criminalite'] = Crim
    X_ajout['pprn'] = Cpprn
    X_ajout['chomage'] = Cchomage
    X_ajout['rev_med'] = Crevenus
    X_ajout['ft_22_categ'] = Cft22
    X_ajout['log_sociaux']=Clog_sociaux
            
    return(X_ajout, y)



In [0]:
def prep_test(X, path):
    #Importation des fichiers de données
    cat = path +'/catastrophes naturelles/'
    crim = path +'criminalité/'
    catnat = pd.read_csv(cat+"catnat_propre.csv", header=0,sep=';')
    crimes = pd.read_excel(crim+"crimes_propre.xlsx", header=0,sep=';')
    pprn = pd.read_excel(cat+"pprn_propre.xlsx", header=0)
    pluie = pd.read_excel(cat+"pluie.xlsx", header=0)
    chomage = pd.read_csv(path+"chomage/chomage_propre.csv", header=0, index_col=0)
    log_sociaux=pd.read_excel(path+"log_sociaux.xls",header=0,sep='')
    revenus = [] #Liste de DataFrames
    revenus_dep = [] #Revenus par départements
    for annee in range(2012,2016):
        revenus.append(pd.read_csv(path+"revenus/revenus_%s_propre.csv"%(annee), header=0, index_col=0))
        revenus_dep.append(pd.read_csv(path+"revenus/revenus_%s_dep_propre.csv"%(annee), header=0, index_col=0))

  
    X_ajout = X.copy()
    Catnat = []
    Crim = []
    Cpprn = []
    Cpluie = []
    Csuperficie = []
    Cft22 = []
    Clog_sociaux = []
    
    ##Parcours des lignes pour crimes et catastrophes naturelles
    for x in X_ajout.itertuples():
    
        ##Données sur le nombre de jour de catastrophes naturelles
        if not is_nan(x.Insee):
            a=catnat[x.Insee==catnat.Insee]
            a=a[x.ft_2_categ==a.annee]
            ##On somme les différents arrêtés simulatannés pour la même zone
            Catnat+=[a.num_risque_jo.sum()]
        else:
            Catnat+=[-1]
    
        ##Données sur les PPRN
        if not is_nan(x.Insee):
            a=pprn[x.Insee[:2]==pprn.cod_commune]
            a=a[x.ft_2_categ==a.dat_mise_a_enquete_deb]
            Cpprn+=[len(a)]
        else:
            Cpprn+=[-1]
    
        ##Données sur le nombre de jour avec plus de 1m de précipitation
        if not is_nan(x.Insee):
            a=pluie[x.Insee[:2]==pluie.departement]
            Cpluie+=[a[x.ft_2_categ].iloc[0]]
        else:
            Cpluie+=[-1]
    
        ##Données sur le taux de criminalité
        if not is_nan(x.Insee):
            a=crimes[x.Insee[:2]==crimes.departement]
            a=a[x.ft_2_categ==a.annee]
            Crim+=[a.criminalite.iloc[0]]
        else:
            Crim+=[-1]
            
        #Données manquantes sur superficief
        if is_nan(x.superficief):
            Csuperficie += [-1]
        else:
            Csuperficie += [x.superficief]
            
        #Données manquantes de ft_22_categ
        if is_nan(x.ft_22_categ):
            Cft22 += [-1]
        else:
            Cft22 += [x.ft_22_categ]
                    
        #Données sur le logement sociaux
        if not is_nan(x.Insee):
            a=log_sociaux[x.Insee[:2]==log_sociaux.Dep]
            Clog_sociaux+=[a.Nb_log_sociaux.iloc[0]]
        else:
            Clog_sociaux+=[np.mean(log_sociaux.Nb_log_sociaux)]
            
    ##Chomage
    Cchomage = ajout_chomage(X_ajout, chomage)
    
    ##Revenus
    Crevenus = ajout_revenu(X, revenus, revenus_dep)
    
    #Ajout des valeurs manquantes
    Catnat = replace_missing_by_med(np.array(Catnat))
    Cpprn = replace_missing_by_med(np.array(Cpprn))
    Cpluie = replace_missing_by_mean(np.array(Cpluie))
    Crim = replace_missing_by_mean(np.array(Crim))
    X_ajout['superficief'] = replace_missing_by_mean(np.array(Csuperficie))
    
    #Ajout des colonnes dans le DataFrame
    X_ajout['pluie'] = Cpluie
    X_ajout['catnat'] = Catnat
    X_ajout['criminalite'] = Crim
    X_ajout['pprn'] = Cpprn
    X_ajout['chomage'] = Cchomage
    X_ajout['rev_med'] = Crevenus
    X_ajout['ft_22_categ'] = Cft22
    X_ajout['log_sociaux']=Clog_sociaux
            
    return X_ajout



# Création du nouveau fichier

Sauvegarde du fichier :

In [0]:
mypath = "/Users/Benjamin/Documents/MATHAPPLI/APST-ML/DataChallenge/DataChallengeGenerali/data/"
#mypath= "C:/Users/Asus/Documents/Cours/EI2/APST/Generali/"

In [0]:
X_train_new_data, y_train_new_data = prep_train(X_train, y_train, mypath)
X_train_new_data.to_csv(mypath+'X_train_new_data.csv')
y_train_new_data.to_csv(mypath+'y_train_saegPGl_new_data.csv')

In [0]:
X_test_new_data = prep_test(X_test, mypath)
X_test_new_data.to_csv(mypath+'X_test_new_data.csv')